In [4]:
import pandas as pd
import os
import re
import nltk
import numpy as np
import time
os.chdir(r"C:\Users\csamassa\Desktop\Mémoire\Nouveau GDN")

In [ ]:
demo=pd.read_csv("DEMOCRATIE_ET_CITOYENNETE.csv", 
                 sep=",",
                 #nrows=10,
                 usecols=[0,10,2,11,13,14,16,17,19,20,22,23,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40,42,43,44,45,46,47]
                 ,dtype={"authorZipCode":object}
                 )
fisc=pd.read_csv("LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.csv", 
                 sep=",",
                 #nrows=10,
                 usecols=[0,10,2,11,12,13,14,15,16,17,18]
                 ,dtype={"authorZipCode":object}
                 )
eco=pd.read_csv("LA_TRANSITION_ECOLOGIQUE.csv", 
                sep=",",
                #nrows=10,               
                usecols=[0,10,2,11,12,14,16,17,18,20,22,23,24,25,26]
                ,dtype={"authorZipCode":object}
                )
                
org=pd.read_csv("ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.csv", 
                sep=",",
               #nrows=10,
                usecols=[0,10,2,11,13,15,16,19,20,21,24,25,27,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43]
                ,dtype={"authorZipCode":object}

In [ ]:
# On rajoute une colonne comportant le thème de chaque question
demo.insert(column="Thème",value="DEMOCRATIE ET CITOYENNETE",loc=3)
fisc.insert(column="Thème",value="LA FISCALITE ET LES DEPENSES PUBLIQUES",loc=3)
eco.insert(column="Thème",value="LA TRANSITION ECOLOGIQUE",loc=3)
org.insert(column="Thème",value="ORGANISATION DE L'ETAT ET DES SERVICES PUBLIQUES",loc=3)


# On nettoie le début des questions
def clean_question(df):
    colonnes=df.columns
    colonnes1=[re.sub(pattern=r"\bQ[A-Za-z0-9]+\s+\-\s",repl='',string=nom) for nom in colonnes]
    return(colonnes1)



# On applique tt en même temps
demo.columns,fisc.columns,eco.columns,org.columns=clean_question(demo),clean_question(fisc),clean_question(eco),clean_question(org)
col1=["id", "authorZipCode","Thème"]

def empiller(df):
    stack_0=df.loc[:, ~df.columns.isin(col1)].stack(dropna=False) # prend les QO (toutes les questions sauf celles de col1)
    stack_1=stack_0.reset_index()                                 # On supp l'index pour avoir le level 0 pour la future jointure
    stack_2=stack_1.merge(df[col1],left_on="level_0",right_index=True,how="left")
    stack_2.columns=["idx_0","Question","Réponse","id","authorZipCode","Thème"] #idx_0 c'est le numéro de la ligne dans le fichier original de chaque thème 
    stack_2.dropna(inplace=True)
    return stack_2


demo_1,fisc_1,eco_1,org_1=empiller(demo),empiller(fisc),empiller(eco),empiller(org)

del(demo,fisc,eco,org)# On supp les variables inutiles de l'environnement

contributions_emp=pd.concat([demo_1,fisc_1,eco_1,org_1],axis=0)   #On met tout dans un même df
 
contributions_emp.reset_index(drop=True,inplace=True)

del(demo_1,fisc_1,eco_1,org_1,col1)

contributions_emp.drop(columns="id", inplace=True) # On supp la colonne (on a qu'à utiliser idx_0 si on veut la trace des contributions)

contributions_emp.to_csv('contributions.csv')


Pas besoin de réexecuter la partie du dessus (long à l'execution et gourmand en mémoire), il suffit d'importer le fichier csv qu'on a exporté

In [2]:
start_time = time.time()

contributions=pd.read_csv('contributions.csv')

print("--- %s seconds ---" % (time.time() - start_time))

In [6]:
start_time = time.time()

def compte_group(df,nom,fichier=None,export=False):
    resultat=df.loc[df["Réponse"].str.contains(nom,case=False, regex=True)]
    ###On va mettre chaque thème dans une feuille différente
    ##D'abord on filtre les résultats par thèmes
   
    if export==True:
        resultat_1=resultat.loc[resultat["Thème"]=="DEMOCRATIE ET CITOYENNETE"]
        resultat_2=resultat.loc[resultat["Thème"]=="LA FISCALITE ET LES DEPENSES PUBLIQUES"]
        resultat_3=resultat.loc[resultat["Thème"]=="LA TRANSITION ECOLOGIQUE"]
        resultat_4=resultat.loc[resultat["Thème"]=="ORGANISATION DE L'ETAT ET DES SERVICES PUBLIQUES"] # je sais que c'est public mais changer ça est trop chiant à faire pzrce que faudra changer le nom des thèmes (ou y'a la faute aussi)
    
        writer = pd.ExcelWriter(fichier+'.xlsx', engine='xlsxwriter')
    
        resultat_1.to_excel(writer, sheet_name="DEMOCRATIE")
        resultat_2.to_excel(writer, sheet_name="FISCALITE")
        resultat_3.to_excel(writer, sheet_name="TRANSITION_ECOLOGIQUE")
        resultat_4.to_excel(writer, sheet_name="ORGANISATION_DE_LETAT") 
    
        writer.save()
    return(resultat)

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.0 seconds ---


On va prendre un exemple pour essayer notre code: toutes les contributions du GDN qui contiennent les mots pandémie, épidémie sras, coronavirus

In [7]:
start_time = time.time()
corona=compte_group(contributions,r"\bpand[e-é]mi[a-z]\b|\b[e-é]pid[e-é]mie\b|\bsras\b|\bcoronavirus\b")

aleo=corona["Réponse"].apply(nltk.sent_tokenize)

#DF series.apply est plus rapide que df.apply

#On ne  prend que les contributions de plus d'une phrase
essai=pd.DataFrame(aleo.loc[aleo.apply(len)>1].apply(pd.Series).stack()).reset_index()#la fonction series fait que le séparateur devient la virgule entre chaque éléments de la liste, le stack fait qu'on les empile, on a un multi index où chaque index de level 0= numéro de la contribution, level 1= numéro de la phrase dans la contribution

#On ajoute le nombre de phrases de chaque contributions
essai_gpby=essai.groupby(by="level_0",as_index=False)[0].count()

essai=essai.merge(essai_gpby,on="level_0")


essai.columns=["level_0","nb_sent","sent","nb_sent_total"]
contain=essai.loc[essai["sent"].str.contains(r"\bpand[e-é]mi[a-z]\b|\b[e-é]pid[e-é]mie\b|\bsras\b|\bcoronavirus\b",regex=True,case=False)]

# On met dans un dictionnaire l'index de chaque phrase contenant notre pattern
dico_lvl_contain=dict(zip(contain.index,zip(contain["nb_sent"],contain["nb_sent_total"])))

print("--- %s seconds ---" % (time.time() - start_time))


--- 68.04011678695679 seconds ---


Pour chaque phrase qui contient les mots qu'on veut, on prend celle d'avant et celle d'après. 
Si jamais la phrase qui contient les patterns est la première, python va renvoyer une erreur lorsqu'on va lui demander 
de prendre la phrase d'avant vu qu'il n'y a rien avant, viceversa si c'est après. 
Pour résoudre cela, on va dire à python d'exclure la première lorsqu'on lui demandera de prendre les phrases x-1 et la dernière phrase quand on lui demande de prendre les phrases x+1 (encore une fois si y'a rien après x on va avoir une erreur index out of bounds). 
On a selectionné dans une autre opérationles phrases qui contiennent le pattern, cette opération a pour but de prendre x-1 et x+1 pas x. 
Si la première phrase contient le pattern, python ne va prendre que celle qui la suit, et si dernière phrase contient le pattern, python ne va prendre que celle qui la précède, 
Si la première et la deuxième phrase contiennent le patern, python va prendre celle qui la suit (x+1), et la ligne suivante python va de nouveau prendre la première (x-1, qui contient le pattern),  et la troisème (x+1), la deuxième étant x sur cette ligne (x+1 la ligne précédente)

In [9]:
start_time = time.time()

idx_avant=[[k-1,k,v] for (k,v) in dico_lvl_contain.items() if v[0]>0] #prend la phrase d'avant si la phrase  (celle qui matche le pattern) n'est pas la première de la contribution, donc d'index supérieur à 0
phrase_avant=essai.loc[[x[0] for x in idx_avant],]
phrase_avant["id_x"]=essai.loc[[x[1] for x in idx_avant],].index # On ajoute l'index des phrases qui contiennent la phrase du patern(celle d'après du coup si répond aux conditions du for)
phrase_avant.columns=["level_0","nb_sent_av", "sent_av","nb_sent_total","id_x"]

idx_apres=[[k+1,k,v] for (k,v) in dico_lvl_contain.items() if v[0]<(v[1]-1)] # # prend la phrase d'après si la phrase n'est pas la dernière de la contribution, donc d'index inférieure au nombre de phrases dans la contribution. on ajoute moins 1 parce que v[1] c'est la taille par ex 20 et v[0] c'est l'index (0:19) donc faut mettre v[1]-1 pour que ça corresponde

phrase_apres=essai.loc[[x[0] for x in idx_apres],]
phrase_apres["id_x"]=essai.loc[[x[1] for x in idx_apres],].index #On ajoute l'index des phrases du pattern (celle d'avant du coup si matche les conditions)
phrase_apres.columns=["level_0","nb_sent_ap", "sent_ap","nb_sent_total","id_x"]

contain=contain.rename_axis('id_x').reset_index() #On ajoute la colonne de l'id pour la jointure
essai_final=contain.merge(phrase_avant, on="id_x", how="outer").merge(phrase_apres,on="id_x",how="outer") #On fait une union des trois dataframes
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.03500008583068848 seconds ---


In [13]:
essai_final.loc[:,["sent","sent_av","sent_ap"]]

,sent,sent_av,sent_ap
0,"Lutte contre EPIDEMIE sida, diminution nb d’av...",Que tous les profs les informent de l’existenc...,Tous les enfants ont les mêmes droits.
1,Prévue pour faire face aux risques d'épidémie ...,qui est une injustice sociale exorbitante au r...,Elle donne donc lieu à une immigration sanitai...
2,Si leur dossier est recevable parce que la gue...,"Ensuite la langue, les lois, les DEVOIRS.",Si comme dirait le Premier Ministre Néozélanda...
3,Il faut pourvoir distinguer les asiles d'urgen...,NaN,"Pour les autres, il faut faire appliquer des c..."
4,"Comme l'épidémie n'a pas eu lieu, les Français...","Par exemple, l'histoire du vaccin pandémique.","Par contre, il faut que l'état soit exemplaire..."
...,...,...,...
163,"le taux d’absentéisme pour raisons de santé, q...",Le rétablissement du jour de carence suffirait...,le problème tient au statut très avantageux de...
164,"aussi des congés, une épidémie et des services...","ça ne se passe pas si aml que ça, sauf que de ...",NaN
165,Lorsqu’on a besoin d’une consultation simple (...,Les cabinets de médecins sont débordés et la p...,Il s’agirait d’un niveau « 1 » qui desorgorger...
166,Les pénuries de médicaments de plus en plus fr...,NaN,Il faut assurer notre indépendance dans ce dom...
